In [24]:
import os
from datetime import datetime

# Bugünün tarihi (örnek veri için)
today_str = datetime.today().strftime('%Y%m%d')

# Ana klasör
base_path = 'NamasteKart'

# Alt klasörler
folders = [
    f"{base_path}/incoming_files/{today_str}",
    f"{base_path}/success_files/{today_str}",
    f"{base_path}/rejected_files/{today_str}"
]

# Klasörleri oluştur
for folder in folders:
    os.makedirs(folder, exist_ok=True)

print("Klasör yapısı oluşturuldu.")


Klasör yapısı oluşturuldu.


In [26]:
import pandas as pd
import os

# Bugünün tarihi
today_str = datetime.today().strftime('%Y%m%d')

# Dosya yolları
base_path = 'NamasteKart'
incoming_path = os.path.join(base_path, 'incoming_files', today_str)

# Dosyaları oku
orders_file = os.path.join(incoming_path, 'orders.csv')
product_master_file = os.path.join(incoming_path, 'product_master.csv')
error_orders_file = os.path.join(incoming_path, 'error_orders.csv')  # örnek hata testi için

# Dosyaları pandas ile oku
orders_df = pd.read_csv(orders_file)
product_master_df = pd.read_csv(product_master_file)
error_orders_df = pd.read_csv(error_orders_file)

# İlk 5 satırı göster (örnek)
print("Orders:")
print(orders_df.head())

print("\nProduct Master:")
print(product_master_df.head())


Orders:
   order_id  order_date  product_id  quantity  sales       city
0         1  2023-01-01         100         1  50000  Bangalore
1         2  2023-02-01         200         2   9000     Mumbai
2         3  2023-01-03         200         2   4500     Mumbai
3         4  2023-01-04         500         3  30000  Bangalore

Product Master:
   product_id product_name  price     category
0         100       iphone  50000       mobile
1         200        chair   4500    furniture
2         300        table  10000    furniture
3         400           tv  25000  electronics
4         500       fridge  10000  electronics


In [27]:
import pandas as pd
from datetime import datetime

def validate_orders(orders_df, product_df):
    error_rows = []
    current_date = datetime.now().date()

    for index, row in orders_df.iterrows():
        reasons = []

        # 1. Product ID product master'da var mı?
        if row['product_id'] not in product_df['product_id'].values:
            reasons.append("Invalid product_id")

        # 2. Satış tutarı doğru mu?
        product_info = product_df[product_df['product_id'] == row['product_id']]
        if not product_info.empty:
            expected_sales = int(product_info.iloc[0]['price']) * int(row['quantity'])
            if expected_sales != int(row['sales']):
                reasons.append("Incorrect sales amount")
        else:
            reasons.append("Missing product info")

        # 3. Gelecekteki bir tarih mi?
        try:
            order_date = pd.to_datetime(row['order_date']).date()
            if order_date > current_date:
                reasons.append("Order date is in the future")
        except:
            reasons.append("Invalid order_date format")

        # 4. Boş alan var mı?
        if row.isnull().any() or (row.astype(str) == '').any():
            reasons.append("Empty field(s) found")

        # 5. Şehir doğru mu?
        if row['city'] not in ['Mumbai', 'Bangalore']:
            reasons.append("Invalid city")

        # Hata varsa ekle
        if reasons:
            error_row = row.to_dict()
            error_row['rejection_reason'] = '; '.join(reasons)
            error_rows.append(error_row)

    return pd.DataFrame(error_rows)


In [28]:
error_orders_df = validate_orders(orders_df, product_master_df)


In [29]:
import os
import pandas as pd
from datetime import datetime

def validate_orders(orders_df, product_df):
    error_rows = []
    current_date = datetime.now().date()

    for _, row in orders_df.iterrows():
        reasons = []

        # 1. Product ID kontrolü
        if row['product_id'] not in product_df['product_id'].values:
            reasons.append("Invalid product_id")

        # 2. Satış tutarı kontrolü
        product_info = product_df[product_df['product_id'] == row['product_id']]
        if not product_info.empty:
            expected_sales = int(product_info.iloc[0]['price']) * int(row['quantity'])
            if expected_sales != int(row['sales']):
                reasons.append("Incorrect sales amount")

        # 3. Gelecek tarih kontrolü
        try:
            order_date = pd.to_datetime(row['order_date']).date()
            if order_date > current_date:
                reasons.append("Order date is in the future")
        except:
            reasons.append("Invalid order_date format")

        # 4. Boş alan kontrolü
        if row.isnull().any() or (row.astype(str) == '').any():
            reasons.append("Empty field(s) found")

        # 5. Şehir kontrolü
        if row['city'] not in ['Mumbai', 'Bangalore']:
            reasons.append("Invalid city")

        if reasons:
            error_row = row.to_dict()
            error_row['rejection_reason'] = '; '.join(reasons)
            error_rows.append(error_row)

    return pd.DataFrame(error_rows)

def process_orders(base_path):
    today_str = datetime.now().strftime('%Y%m%d')
    incoming_path = os.path.join(base_path, 'incoming_files', today_str)
    success_path = os.path.join(base_path, 'success_files', today_str)
    rejected_path = os.path.join(base_path, 'rejected_files', today_str)

    os.makedirs(success_path, exist_ok=True)
    os.makedirs(rejected_path, exist_ok=True)

    files = [f for f in os.listdir(incoming_path) if f.endswith('.csv')]

    summary = {'total': 0, 'success': 0, 'rejected': 0}

    for file in files:
        summary['total'] += 1
        orders_df = pd.read_csv(os.path.join(incoming_path, file))
        product_df = pd.read_csv(os.path.join(incoming_path, 'product_master.csv'))

        errors_df = validate_orders(orders_df, product_df)

        if errors_df.empty:
            # Başarılı dosya
            orders_df.to_csv(os.path.join(success_path, file), index=False)
            summary['success'] += 1
        else:
            # Hatalı dosya ve hata nedeni
            orders_df.to_csv(os.path.join(rejected_path, file), index=False)
            error_file_name = f'error_{file}'
            errors_df.to_csv(os.path.join(rejected_path, error_file_name), index=False)
            summary['rejected'] += 1

    print(f"Validation complete: {summary['total']} files processed: {summary['success']} success, {summary['rejected']} rejected.")


In [30]:
print(error_orders_df)
print(f"Hatalı sipariş sayısı: {len(error_orders_df)}")


   order_id  order_date  product_id  quantity  sales    city  \
0         3  2023-01-03         200         2   4500  Mumbai   

         rejection_reason  
0  Incorrect sales amount  
Hatalı sipariş sayısı: 1


In [31]:
orders_df.loc[0, 'sales'] = 99999  # Satış tutarını boz
error_orders_df = validate_orders(orders_df, product_master_df)
print(error_orders_df)
print(f"Hatalı sipariş sayısı: {len(error_orders_df)}")


   order_id  order_date  product_id  quantity  sales       city  \
0         1  2023-01-01         100         1  99999  Bangalore   
1         3  2023-01-03         200         2   4500     Mumbai   

         rejection_reason  
0  Incorrect sales amount  
1  Incorrect sales amount  
Hatalı sipariş sayısı: 2


In [32]:
import os
import pandas as pd
from datetime import datetime
from shutil import copyfile

# 1. GÜNÜN TARİHİNİ AL VE KLASÖRLERİ AYARLA
today_str = datetime.now().strftime('%Y%m%d')
base_dir = 'NamasteKart'
incoming_path = os.path.join(base_dir, 'incoming_files', today_str)
success_path = os.path.join(base_dir, 'success_files', today_str)
rejected_path = os.path.join(base_dir, 'rejected_files', today_str)

# Klasörleri oluştur
os.makedirs(success_path, exist_ok=True)
os.makedirs(rejected_path, exist_ok=True)

# 2. TÜM CSV DOSYALARINI OKU
incoming_files = [f for f in os.listdir(incoming_path) if f.endswith('.csv')]

# 3. ÜRÜN MASTER DOSYASINI YÜKLE
product_master_df = pd.read_csv(os.path.join(incoming_path, 'product_master.csv'))

# 4. HER DOSYAYI TEK TEK İŞLE
for file in incoming_files:
    if file == 'product_master.csv':
        continue  # bu dosya validation için kullanılıyor

    orders_df = pd.read_csv(os.path.join(incoming_path, file))
    error_orders_df = validate_orders(orders_df, product_master_df)

    if error_orders_df.empty:
        # Hatalı satır yoksa başarıyla taşı
        copyfile(os.path.join(incoming_path, file), os.path.join(success_path, file))
        print(f"{file} -> ✅ success_files")
    else:
        # Dosya hatalı: rejected klasörüne taşı ve error_ dosyasını oluştur
        copyfile(os.path.join(incoming_path, file), os.path.join(rejected_path, file))

        # error_ dosyası oluştur
        error_filename = f"error_{file}"
        error_orders_df.to_csv(os.path.join(rejected_path, error_filename), index=False)
        print(f"{file} -> ❌ rejected_files + error_{file}")


orders.csv -> ❌ rejected_files + error_orders.csv
error_orders.csv -> ❌ rejected_files + error_error_orders.csv


In [33]:
from datetime import datetime
import os

def prepare_email_summary(base_dir='NamasteKart'):
    today_str = datetime.now().strftime('%Y%m%d')
    
    incoming_path = os.path.join(base_dir, 'incoming_files', today_str)
    success_path = os.path.join(base_dir, 'success_files', today_str)
    rejected_path = os.path.join(base_dir, 'rejected_files', today_str)

    # Dosya sayımlarını yap
    incoming_files = [f for f in os.listdir(incoming_path) if f.endswith('.csv') and not f.startswith('error_') and f != 'product_master.csv']
    success_files = [f for f in os.listdir(success_path) if f.endswith('.csv')]
    rejected_files = [f for f in os.listdir(rejected_path) if f.endswith('.csv') and not f.startswith('error_')]

    # Özet verileri
    total_incoming = len(incoming_files)
    total_success = len(success_files)
    total_rejected = len(rejected_files)

    # E-posta içeriği
    subject = f"Validation Email {today_str}"
    body = (
        f"Hi Team,\n\n"
        f"Validation Summary for {today_str}:\n"
        f"- Total Incoming Files: {total_incoming}\n"
        f"- Successful Files: {total_success}\n"
        f"- Rejected Files: {total_rejected}\n\n"
        f"Regards,\nNamasteKart QA Team"
    )

    print(f"Subject: {subject}")
    print("\n" + body)

# Fonksiyonu çağır
prepare_email_summary()


Subject: Validation Email 20250502

Hi Team,

Validation Summary for 20250502:
- Total Incoming Files: 1
- Successful Files: 0
- Rejected Files: 1

Regards,
NamasteKart QA Team
